# Lesson 2: solving Markov chains

In [1]:
import marmote.core as mc
import marmote.markovchain as mmc
import numpy as np

In Lesson1, we have seen how to create and inspect Markov chains. We now illustrates the different metrics that can be computed on them
(what is called "the solution").

## First example: solving discrete-time Markov chains

We first (re)create the 3-state discrete-time Markov chain of Lesson 1

In [2]:
states = np.array( [0, 1, 2] )
n = states.shape[0]
P = mc.FullMatrix(n)
P.set_type(mc.DISCRETE)
P.setEntry(0,0,0.25)
P.setEntry(0,1,0.5)
P.setEntry(0,2,0.25)
P.setEntry(1,0,0.4)
P.setEntry(1,1,0.2)
P.setEntry(1,2,0.4)
P.setEntry(2,0,0.4)
P.setEntry(2,1,0.3)
P.setEntry(2,2,0.3)
initial_prob = np.array( [0.2, 0.2, 0.6] )
initial = mc.DiscreteDistribution(states, initial_prob)

In [3]:
c1 = mmc.MarkovChain( P )
c1.set_init_distribution(initial)
c1.set_model_name( "Demo" )

### Transient distributions

To compute transient distributions, the method to be called is `TransientDistributionDT` with argument the 'time' or number of steps.

In [4]:
pi1 = c1.TransientDistributionDT( 1 )
pi2 = c1.TransientDistributionDT( 2 )
pi3 = c1.TransientDistributionDT( 3 )

In [5]:
print( pi1 )
print( pi2 )
print( pi3 )

Discrete distribution values { 0  1  2  } probas {     0.37     0.32     0.31 }
Discrete distribution values { 0  1  2  } probas {   0.3445    0.342   0.3135 }
Discrete distribution values { 0  1  2  } probas {   0.3483   0.3347    0.317 }


It is of course possible to change the initial distribution. Other distributions of the `DiscreteDistribution` family can be used.<br>
A typical one is `DiracDistribution`.

In [6]:
pi0 = mc.DiracDistribution(0)
c1.set_init_distribution(pi0)
pi1 = c1.TransientDistributionDT( 1 )
pi2 = c1.TransientDistributionDT( 2 )
pi3 = c1.TransientDistributionDT( 3 )
print( pi0 )
print( pi1 )
print( pi2 )
print( pi3 )

Dirac distribution at 0
Discrete distribution values { 0 1 2 } probas {     0.25      0.5     0.25 }
Discrete distribution values { 0  1  2  } probas {   0.3625      0.3   0.3375 }
Discrete distribution values { 0  1  2  } probas {   0.3456   0.3425   0.3119 }


Another possibility is `UniformDiscreteDistribution`.

In [7]:
pi0 = mc.UniformDiscreteDistribution(0,2)
c1.set_init_distribution(pi0)
pi1 = c1.TransientDistributionDT( 1 )
pi2 = c1.TransientDistributionDT( 2 )
pi3 = c1.TransientDistributionDT( 3 )
print( pi0 )
print( pi1 )
print( pi2 )
print( pi3 )

Discrete uniform distribution on [0..2]
Discrete distribution values { 0  1  2  } probas {     0.35   0.3333   0.3167 }
Discrete distribution values { 0  1  2  } probas {   0.3475   0.3367   0.3158 }
Discrete distribution values { 0  1  2  } probas {   0.3479   0.3358   0.3163 }


### Stationary distribution

Computing the stationary distribution of a Markov chain is a typical activity of the Markov modeler.
`Marmote` provides several ways to perform this task.

There exists a default method `StationaryDistribution()` for users who don't want to bother about details.

In [8]:
pista = c1.StationaryDistribution()

In [9]:
print(pista)

Discrete distribution values { 0  1  2  } probas {   0.3478    0.336   0.3162 }


This is an iterative and approximate method. To see that the result is not exact, use the `TransientDistribution()` method to perform one step of the transition matrix. Then compute the distance between the two distributions.

In [10]:
c1.set_init_distribution( pista )
dis = c1.TransientDistributionDT(1)
print( "Distance between pi and pi.P:", mc.DiscreteDistribution.DistanceL1( pista, dis ) )

Distance between pi and pi.P: 3.409902343820548e-08


`Marmote` provides an improved iterative method, the **red light green light** algorithm recently published by Brown, Avrachenkov and Nitvak.

In [11]:
pista2 = c1.StationaryDistributionRLGL( 100, 1e-10, mc.UniformDiscreteDistribution(0,2), False )

In [12]:
mc.DiscreteDistribution.DistanceL1( pista, pista2 )

2.962278516926986e-08

Of course, for such a small example, the exact distribution can be computed.
Compare the approximate solution with the exact one.

In [13]:
prosta_ex = np.array( [ 8/23, 85/253, 80/253 ], dtype=float )
pista_ex = mc.DiscreteDistribution( states, prosta_ex )
print(pista_ex)

Discrete distribution values { 0 1 2 } probas {   0.3478    0.336   0.3162 }


In [14]:
print( "Distance between numerical pi (standard) and exact pi:", mc.DiscreteDistribution.DistanceL1( pista, pista_ex ) )
print( "Distance between numerical pi (RLGL) and exact pi:", mc.DiscreteDistribution.DistanceL1( pista2, pista_ex ) )

Distance between numerical pi (standard) and exact pi: 2.963156187085758e-08
Distance between numerical pi (RLGL) and exact pi: 2.457783976339556e-11


Finally, check that the exact stationary distribution is exact.

In [15]:
c1.set_init_distribution(pista_ex)
didi = c1.TransientDistributionDT(1)

In [16]:
mc.DiscreteDistribution.DistanceL1( pista_ex, didi )

0.0

### Simulation

Simulation has plenty of controls. The basic syntax is `SimulateChain( nb_steps, stats, traj, trace )` where:

* `nb_steps` is the number of time instants to simulate
* `stats` is a boolean specifying if occupancy statistics are to be kept
* `traj` is a boolean specifying if a trajectory is to be kept
* `trace` is a boolean specifying if the trajectory is to be printed along the way.

For instance, a simulation of 10 time steps, no statistics, a trajectory which is not printed.

In [17]:
simRes = c1.SimulateChainDT( 10, stats=False, traj=True, trace=False )

Inspecting the features of the simulation object: there is indeed a DT (discrete-time) trajectory, but no CT (continuous-time) trajectory.

In [18]:
simRes.Diagnose()

# Simulation result
# Time type: discrete
# Keeps a trajectory of size 11
# DT trajectory size: 11
# CT trajectory size: 0
# Last state: 2
# Last time:  10


Listing the details.

In [19]:
print( simRes.DT_dates() )
print( simRes.states() )
print( simRes.lastDate() )
print( simRes.lastState() )

[ 0  1  2  3  4  5  6  7  8  9 10]
[0 2 2 1 0 0 1 0 1 2 2]
10
2


Running again with trajectory printed but not kept, and stats.

Observe the repeated last columns with states. More explanations on this below.

In [20]:
simRes = c1.SimulateChainDT( 10, True, False, True )

         0        1 1
         1        1 1
         2        1 1
         3        2 2
         4        0 0
         5        1 1
         6        1 1
         7        0 0
         8        2 2
         9        0 0
        10        1 1


The empirical distribution can be extracted from the `SimulationResult` object via its method `Distribution()`.
This produces some information on the output and stores the result in a `DiscreteDistribution` object.

In [21]:
trDis = simRes.Distribution()

# State   0:   cum time =        3   (30%)
# State   1:   cum time =        5   (50%)
# State   2:   cum time =        2   (20%)


In [22]:
print( trDis )

Discrete distribution values { 0  1  2  } probas {      0.3      0.5      0.2 }


## Second example: solving continuous-time Markov chains

We first recreate the continuous-time Markov chain of Lesson 1

In [23]:
Q = mc.SparseMatrix(6)
Q.set_type(mc.CONTINUOUS)
Q.setEntry(0,1,1.0)
Q.setEntry(0,0,-1.0)
for i in range(1,6):
    if i > 0:
        Q.setEntry(i,0,1.0)
        Q.addToEntry(i,i,-1.0)
    if i < 5:
        Q.setEntry(i,i+1,1.0)
        Q.addToEntry(i,i,-1.0)

In [24]:
c2 = mmc.MarkovChain( Q )
c2.set_init_distribution(initial)
c2.set_model_name( "Demo_Continuous" )

### State distributions

Standard call to stationary distribution method

In [25]:
stadis = c2.StationaryDistribution()
print(stadis)

Discrete distribution values { 0  1  2  3  4  5  } probas {      0.5     0.25    0.125   0.0625  0.03125  0.03125 }


### Simulation

Simulation is as for discrete-time chains, but there is an additional control:
`withIncrements` specifies whether time increments (sojourn time in each state) should be printed
when tracing is enabled.

Example of a simulation of 10 time units, with all details printed. 
Each line of the output contains:

* the sequence number of the transition event inside square brackets
* the time at which this event occured
* the state *index* that *results* from the transition
* the time increment between is event and the previous one
* the state again, but now in full representation (in this example this is the same as the index).

In [26]:
simres = c2.SimulateChainCT( 10.0, stats=False, traj=True, withIncrements=True, trace=True )

[   0]     0.000000        0   0.000000 0
[   1]     0.704633        1   0.704633 1
[   2]     1.457544        2   0.752910 2
[   3]     1.732233        3   0.274690 3
[   4]     2.232788        0   0.500555 0
[   5]     2.657029        1   0.424241 1
[   6]     2.862473        2   0.205444 2
[   7]     2.867068        3   0.004595 3
[   8]     3.820435        0   0.953367 0
[   9]     4.494710        1   0.674275 1
[  10]     4.498088        0   0.003378 0
[  11]     5.850046        1   1.351958 1
[  12]     6.369734        2   0.519688 2
[  13]     7.016731        0   0.646998 0
[  14]     7.222356        1   0.205625 1
[  15]     7.319826        2   0.097470 2
[  16]     8.074626        3   0.754800 3
[  17]     8.494826        4   0.420200 4
[  18]     8.525130        5   0.030303 5
[  19]     8.878892        0   0.353762 0
[  20]     9.610734        1   0.731843 1
[  21]    10.000000        2   0.389266 2


Observe the following conventions:

* the first event (#0) is always at time 0 and it results in a transition to the initial state;
* the last date is exactly the simulation time specified;
* the sojourn time in the state resulting from event #n is to be read in the following line: that of event #n+1.

The trajectory has been kept. It can be accessed e.g. for post-processing.

In [27]:
print( simres.CT_dates() )
print( simres.states() )

[ 0.          0.70463336  1.45754369  1.73223331  2.23278824  2.65702925
  2.86247279  2.86706784  3.82043494  4.49470957  4.49808781  5.85004578
  6.36973369  7.01673122  7.2223561   7.31982573  8.0746262   8.49482632
  8.52512969  8.87889168  9.61073446 10.        ]
[0 1 2 3 0 1 2 3 0 1 0 1 2 0 1 2 3 4 5 0 1 2]


### Hitting time distributions

For general Markov chains, hitting time distributions can be simulated.
Hitting time methods all require as argument a *hit set indicator* which is an array of booleans where `True` marks the states belonging to the hitting set.

Here, the hitting set contains just the last state (index 5).

In [28]:
hitset = np.array( [ False, False, False, False, False, True ], dtype=bool )


Hitting time simulation methods have several controls.
The basic syntax is `SimulateHittingTime( init, hitset, sample_nb, max_time )` where:

* `init` specifies the initial distribution: either a state number or a `DiscreteDistribution` object
* `hitset` is the hitting set indicator
* `sample_nb` is the number of samples to be drawn
* `max_time` is a time limit for simulations: hitting times larger than this value will not be found.

Example: simulating 25 values of the hitting time from state 0.

In [29]:
simres = c2.SimulateHittingTime( 0, hitset, 25, 100  )

The samples of the simulation are stored in the attribute `CT_dates` of the simulation object.

In [30]:
print( simres.CT_dates() )

[  2.25621908  47.82934132  55.49530805   8.71000108  92.30982759
  22.83794963   7.090968    17.65960161  91.43554553  49.56252067
   4.75619739  63.09539908  19.70872092  58.51844151  48.71590588
  67.90897413  21.00038601  57.54621086  28.2246774    2.77211519
  57.39968514 100.4853776   45.52337486   5.31724674  40.08304546]


It is also possible to compute average hitting times, starting from all states.
The method for this is `AverageHittingTime`. It takes the hitting set as unique argument.

In [31]:
avghit = c2.AverageHittingTime( hitset )
print( avghit )

[31. 30. 28. 24. 16.  0.]


Comparing with the empirical average of the simulation.

In [32]:
avg = 0
for i in range(25):
    avg = avg + simres.CT_dates()[i]
print( "Empirical average of hitting time from state 0 =", avg/25 )

Empirical average of hitting time from state 0 = 40.649721629688365
